In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
data_generated = dataiku.Dataset("data_generated")
data_generated_df = data_generated.get_dataframe()



In [2]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [13]:
del data_generated_df["Timestamp"]
df_cpy  = data_generated_df.set_index('date')

In [7]:
data = df_cpy
scalers={}
for i in df_cpy.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(data[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    data[i]=s_s

/opt/dataiku/code-env/lib64/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
data_to_pred = data.values
data_to_pred = data_to_pred.reshape((1, data_to_pred.shape[0],data_to_pred.shape[1]))

In [9]:
handle = dataiku.Folder("model_forecast")
json_config = handle.read_json("model_json")
model = keras.models.model_from_json(json_config)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [10]:
data_pred = model.predict(data_to_pred)

ValueError: Error when checking input: expected input_1 to have shape (10, 30) but got array with shape (10, 31)

In [11]:
for index,i in enumerate(df_cpy.columns):
    scaler = scalers['scaler_'+i]
    data_pred[:,:,index]=scaler.inverse_transform(data_pred[:,:,index])

NameError: name 'data_pred' is not defined

In [0]:
data_pred = data_pred.squeeze()

In [0]:
df_pred = pd.DataFrame(data_pred, columns=df_cpy.columns)

In [0]:
lastdate = df["date"].max()
listdate = []
for i in range(len(val)):
    lastdate = lastdate + timedelta(minutes=15)
    listdate.append(lastdate)
    
df_pred["date"] = listdate

In [0]:
# Write recipe outputs
data_genrerated_prediction = dataiku.Dataset("data_genrerated_prediction")
data_genrerated_prediction.write_with_schema(df_pred)

In [26]:
import dataikuapi

client = dataikuapi.DSSClient("http://34.142.63.173:11000", "eJIRjKPvY6bYTcsdC1e2d1JUmGbrBIF2")

dss_projects = client.list_project_keys()
print(dss_projects)

['ASLALPHABET', 'CREDITCARDFRAUDDETECTION', 'DKU_TUTORIAL_IMAGE_CLASSIFICATION_THE_VISUAL_WAY', 'LOGSANALYSIS']


In [27]:


project = client.get_project("LOGSANALYSIS")

# Read recipe inputs
data_generated = project.get_dataset("data_generated")

In [31]:
data_generated.get_as_core_dataset().get_dataframe()

,date,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
0,2021-08-25 10:30:00+00:00,13.878968,12.286661,13.531464,11.483321,7.426455,3.422888,3.473716,80.051149,74.255877,73.992872,67.673485,82.512308,0.250618,0.222704,13.983509,13.192545,9.004598,11.065908,7.846581,3.432616,3.448435,3.121033,80.356122,75.704022,66.847742,67.747304,86.366592,0.290478,0.214987,0.201346
1,2021-08-25 10:15:00+00:00,13.878987,12.365145,13.530892,11.483326,7.563097,3.423088,3.473949,80.001296,74.314085,73.967484,67.817156,82.583297,0.250618,0.222704,13.983552,13.119092,8.945338,10.987782,7.846591,3.432626,3.448440,3.121042,80.364964,75.660955,66.849678,67.866895,86.501105,0.290478,0.214987,0.201346
2,2021-08-25 10:00:00+00:00,13.879335,12.365150,13.452938,11.483335,7.494743,3.422978,3.473935,79.856576,74.252812,73.914726,67.866273,82.604449,0.250618,0.222704,14.000411,13.114810,9.082032,10.948824,7.846596,3.432888,3.448526,3.121057,80.328325,75.623843,66.847866,67.891991,86.435144,0.290478,0.214987,0.201346
3,2021-08-25 09:45:00+00:00,13.761989,12.247784,13.530916,11.405381,7.494747,3.423260,3.473916,79.964993,74.342012,73.813582,67.758075,82.604141,0.250618,0.222704,13.883098,13.153267,9.082032,10.988097,7.846939,3.432950,3.448612,3.121066,80.351475,75.612728,66.742057,67.809923,86.460025,0.290478,0.214987,0.201346
4,2021-08-25 09:30:00+00:00,13.879035,12.365188,13.492058,11.483688,7.455794,3.423059,3.474112,79.919106,74.265015,73.817755,67.748589,82.815897,0.250618,0.222704,13.922056,13.231608,9.004464,10.987997,7.846624,3.432993,3.448645,3.121076,80.281120,75.634377,66.740680,67.886144,86.505593,0.290478,0.214987,0.201346
5,2021-08-25 09:15:00+00:00,13.879039,12.267380,13.423584,11.483698,7.573240,3.423760,3.474112,79.908035,74.239338,73.822127,67.602919,82.696222,0.250618,0.222704,13.922104,13.163134,8.965582,11.065961,7.709992,3.432821,3.451297,3.121081,80.232120,75.604229,66.806915,67.834427,86.494110,0.290478,0.214987,0.201346
6,2021-08-25 09:00:00+00:00,13.879388,12.384416,13.570375,11.405534,7.455870,3.423751,3.474374,79.980129,74.312247,73.927719,67.754696,82.718701,0.250618,0.222704,13.863607,13.172954,8.964896,10.929233,7.710001,3.433475,3.451311,3.121248,80.252980,75.669962,66.798292,67.849045,86.538465,0.290478,0.214987,0.201346
7,2021-08-25 08:45:00+00:00,13.879063,12.267227,13.492101,11.405500,7.573255,3.423760,3.474383,80.002633,74.230720,73.892358,67.741353,82.670960,0.250618,0.222704,13.844693,13.231469,9.082442,10.911144,7.729440,3.433451,3.451416,3.121248,80.266723,75.646700,66.727455,67.795677,86.462496,0.290478,0.214987,0.201346
8,2021-08-25 08:30:00+00:00,13.761884,12.384445,13.453052,11.366332,7.572940,3.423956,3.474627,80.063592,74.307682,73.823947,67.783033,82.738510,0.250618,0.222704,13.922251,13.192569,9.003997,11.144125,7.768522,3.433556,3.451406,3.121267,80.368354,75.640119,66.763477,68.001545,86.581014,0.290478,0.214987,0.201346
9,2021-08-25 08:15:00+00:00,13.727762,12.306319,13.570365,11.332057,7.514343,3.423946,3.474608,79.992337,74.362501,73.792685,67.672994,82.619744,0.250618,0.222704,14.000435,13.173011,9.004188,10.997692,7.846672,3.433642,3.451411,3.121276,80.404808,75.600462,66.672579,67.822422,86.517581,0.290478,0.214987,0.201346
